In [23]:
root_path = '/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/'

In [26]:
import numpy as np
import pickle
import pandas as pd
import statsmodels.api as sm
import hdf5storage
from scipy.signal import butter, lfilter
import os, glob, time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from datetime import datetime
from sklearn.externals import joblib
from scipy import signal
import matplotlib.pyplot as plt
import random
    
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
            
        return [Epochs, Num[0]]

def Convert_to_featureVector(EpochsT, NumT, EpochsN, NumN, featureNum):
        FeaturesT = np.zeros((NumT, featureNum))
        for i in range(NumT):
            FeaturesT[i,:] = np.reshape(EpochsT[i,:,:],(1,featureNum))
        FeaturesN = np.zeros((NumN, featureNum))
        for j in range(NumN):
            FeaturesN[j,:] = np.reshape(EpochsN[j,:,:],(1,featureNum))
        return [FeaturesT,FeaturesN]

def Balancing_DataSet(Epochs, size):
    Epochs_New = np.zeros((size, Epochs.shape[1], Epochs.shape[2]))
    
    index = np.random.choice(Epochs.shape[0], size = size, replace = False)
    
    Epochs_New = Epochs[index, :, :]
    
    return Epochs_New
    
def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs

def Make_Average_Component(EpochsT, NumT, EpochsN, NumN, channelNum, epochSampleNum, componentNum):
    EpochsT = Standardization(EpochsT)
    EpochsN = Standardization(EpochsN)
    
    NumT_Aver = NumT-componentNum
    NumN_Aver = NumN-componentNum
    
    EpochsT_Aver = np.zeros((NumT_Aver, channelNum, epochSampleNum))
    EpochsN_Aver = np.zeros((NumN_Aver, channelNum, epochSampleNum))
    for i in range(NumT_Aver):
        EpochsT_Aver[i, :, :] = np.mean(EpochsT[i:i+componentNum, :, :], axis=0)
    for j in range(NumN_Aver):
        EpochsN_Aver[j, :, :] = np.mean(EpochsN[j:j+componentNum, :, :], axis=0)
        
    return [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver]
    
def main():
        start = time.time()
        
        ##Generate Preprocessing Training data
        ctime = datetime.today().strftime("%m%d_%H%M")
        Classifier_path = '/Users/hyuns/Desktop/LDAModel/' + ctime + 'Classifier.pickle'
        channelNum = 7
        
        mat_path = root_path + 'Training/'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime)
        
        matfile_name = current_list[0]
        
        mat = hdf5storage.loadmat(matfile_name)
        channelNames = mat['channelNames']
        eegData = mat['eegData']
        samplingFreq = mat['samplingFreq']
        samplingFreq = samplingFreq[0,0]
        stims = mat['stims']
        channelNum = channelNames.shape
        channelNum = channelNum[1]
        eegData = np.transpose(eegData)
        
        ##Preprocessing process
        sampleNum = eegData.shape[1]

        #Bandpass Filter
        eegData = butter_bandpass_filter(eegData, 0.1, 30, samplingFreq, order=4)
    
        #Epoching
        epochSampleNum = int(np.floor(1.0 * samplingFreq))
        offset = int(np.floor(0.0 * samplingFreq)) 
        baseline = int(np.floor(1.0 * samplingFreq)) 
        [EpochsT, NumT] = Epoching(eegData, stims, 1, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        [EpochsN, NumN] = Epoching(eegData, stims, 0, samplingFreq, channelNum, epochSampleNum, offset, baseline)
        
        NumN = NumT
        EpochsN_New = Balancing_DataSet(EpochsN, NumN)
        
        #Convert to feature vector
        [EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver] = Make_Average_Component(EpochsT, NumT, EpochsN_New, NumN, channelNum, epochSampleNum, 20)
        
        featureNum = channelNum*epochSampleNum
        
        [FeaturesT, FeaturesN] = Convert_to_featureVector(EpochsT_Aver, NumT_Aver, EpochsN_Aver, NumN_Aver, featureNum)
        TrainData = np.concatenate((FeaturesT, FeaturesN))
        TrainLabel = np.concatenate((np.ones((NumT_Aver,1)).astype(int),np.zeros((NumN_Aver,1)).astype(int))).ravel()
        
        #Saving LDA classifier
        lda = LinearDiscriminantAnalysis(solver='lsqr',shrinkage='auto')
        lda.fit(TrainData, TrainLabel)
        joblib.dump(lda, Classifier_path, protocol=2)
        
        print("NumT :", NumT)
        print("NumT_Aver :", NumT_Aver)
        print("NumN_Aver :", NumN_Aver)
        print("time :", time.time() - start)
        
if __name__ == "__main__":
    main()

NumT : 210
NumT_Aver : 190
NumN_Aver : 190
time : 2.3462789058685303


In [27]:
import numpy as np
from scipy.signal import butter, lfilter, sosfiltfilt
import time
import os, glob
import hdf5storage
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.externals import joblib
import shutil
from datetime import datetime
import socket
import matplotlib.pyplot as plt

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

def Standardization(Epochs):
    for i in range(Epochs.shape[1]):
        Epochs[:,i,:] = np.subtract(Epochs[:,i,:], np.mean(Epochs[:,i,:]))
        Epochs[:,i,:] = Epochs[:,i,:] / np.std(Epochs[:,i,:])
    
    return Epochs    

def Make_Average_Component(Epochs, AverSize):
    Num = int(np.floor(Epochs.shape[0]/AverSize))
    
    Epochs_Aver = np.zeros((Num, Epochs.shape[1], Epochs.shape[2]))
    for i in range(Num):
        Epochs_Aver[i, :, :] = np.mean(Epochs[AverSize*i:AverSize*i+AverSize, :, :], axis=0)
    
    return Epochs_Aver
    
def Epoching(eegData, stims, code, samplingRate, nChannel, epochSampleNum, epochOffset,baseline):
        Time = stims[np.where(stims[:,1] == code),0][0]
        Time = np.floor(np.multiply(Time,samplingRate)).astype(int)
        Time_after = np.add(Time,epochOffset).astype(int)
        Time_base = np.add(Time,baseline).astype(int)
        Num = Time.shape
        Epochs = np.zeros((Num[0], nChannel, epochSampleNum))
        for j in range(Num[0]):
            Epochs[j, :, :] = eegData[:, Time_after[j]:Time_after[j] + epochSampleNum]
        
        Epochs = Standardization(Epochs)
        Epochs_Aver = np.mean(Epochs, axis=0)

        return Epochs_Aver

def Convert_to_FeatureVector(Epochs, buttonNum, featureNum):
    Features = np.zeros((buttonNum, featureNum))
    for i in range(buttonNum):
        Features[i, :] = np.reshape(Epochs[i, :, :], (1, featureNum))
    return Features
    
def main():
        Classifier_path = '/Users/hyuns/Desktop/LDAModel/'
        classifier_list = sorted(glob.glob(Classifier_path + '*.pickle'), key=os.path.getmtime, reverse=True)
        print("classifer:", classifier_list[0])
        
        lda = joblib.load(classifier_list[0])
        
        mat_path = root_path + 'Online/'
        current_list = sorted(glob.glob(mat_path + '*.mat'), key=os.path.getmtime, reverse=True)
        score = 0
        
        for mat_file in current_list:
            print(mat_file)
            ans = mat_file[-5]
            
            mat = hdf5storage.loadmat(mat_file)
            channelNames = mat['channelNames']
            eegData = mat['eegData']
            samplingFreq = mat['samplingFreq']
            samplingFreq = samplingFreq[0,0]
            stims = mat['stims']
            channelNum = channelNames.shape
            channelNum = channelNum[1]
            eegData = np.transpose(eegData)
            buttonNum = 7
            
            #Bandpass Filter
            eegData = butter_bandpass_filter(eegData, 0.1, 30, samplingFreq, order=4)

            #Epoching
            epochSampleNum = int(np.floor(1.0 * samplingFreq))
            offset = int(np.floor(0.0 * samplingFreq))
            baseline = int(np.floor(1.0 * samplingFreq))
            
            ####### averaging whole epochs
            Epochs_Aver = np.zeros((buttonNum, channelNum, epochSampleNum))
            
            featureNum = channelNum*epochSampleNum
            
            for i in range(buttonNum):
                Epochs_Aver[i] = Epoching(eegData, stims, (i+1), samplingFreq, channelNum, epochSampleNum, offset, baseline)
         
            Features = Convert_to_FeatureVector(Epochs_Aver, buttonNum, featureNum)
        
            Answers = lda.decision_function(Features)
            answer = np.argmax(Answers) + 1
            
            print(Answers)
            if int(ans) != int(answer): 
                score = score + 1
            print('order: ', ans, 'predict: ', answer)
        
        print('score:', score)
if __name__ == "__main__":
    main()

classifer: /Users/hyuns/Desktop/LDAModel/0803_1052Classifier.pickle
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/Online/1_2.mat
[   10.63265034  1061.96613439 -1238.700827    -819.58518735
  -853.36224125  -632.42317094   136.07007108]
order:  2 predict:  2
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/Online/2_4.mat
[ -607.55806185  -778.05959263  -265.75305457     9.77563682
  -308.77269893 -1002.06219051  -419.81768195]
order:  4 predict:  4
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/Online/3_6.mat
[-185.78610027 -461.70371684 -396.72970583 -474.14384361 -713.7400943
  631.05502756 -379.33912164]
order:  6 predict:  6
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/Online/4_1.mat
[ 426.37940132 -351.30991608 -618.1613359  -471.95620778 -484.36963073
 -446.07938206 -579.38817102]
order:  1 predict:  1
/Users/hyuns/Desktop/HGU/2020-2/Capston

[ -263.92423377   999.19922757  -972.16377357 -1025.7069267
  -803.59755818  -578.93641695  -522.46146126]
order:  2 predict:  2
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/Online/40_7.mat
[-1305.56668557   -18.81452256 -1401.20266979  -742.86130433
  -423.21226856  -458.56110962  1072.18663311]
order:  7 predict:  7
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/Online/41_1.mat
[ 753.71122505 -503.86909451 -615.9102274  -406.43797744 -149.62597663
 -619.8920542  -740.09769237]
order:  1 predict:  1
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/Online/42_2.mat
[  292.13951499   893.80055356  -471.31005067  -424.72739373
  -714.89259879  -783.25604266 -1037.13550047]
order:  2 predict:  2
/Users/hyuns/Desktop/HGU/2020-2/Capstone/Drone Project/EEGData/VR300_Data/0729/hs/Online/43_4.mat
[ -871.32253829 -1148.65061807 -1019.19383861  1235.76902232
  -257.1191508   -747.68306416  -65